In [3]:
import configparser
import warnings
from pymongo import MongoClient

warnings.filterwarnings("ignore", category=DeprecationWarning)

config = configparser.ConfigParser()
config.read('config.ini')
host = config['DEFAULT']['IP']
port = config['DEFAULT']['MongoDB-Port']

client = MongoClient(host, int(port))
col = client['Twitter']['twitter-richard-2014']

In [4]:
terms = [
    'notallmen', 
    'feminazi', 
    'vaw',
    'family violence', 
    'domestic violence', 
    'toxic masculinity',
    'Rosie Batty',
    'OurWatchAus', 
    'freefromviolence', 
    'mrbenjaminlaw',
    'howiwillchange'
]

In [6]:
records = []
for term in terms:
    fetched_records = list(col.find({"$text": {"$search": "\"" + term + "\"", "$caseSensitive": False}}))
    records.extend([record for record in fetched_records if record not in records])

In [25]:
import spacy
spacy.load('en')
from spacy.lang.en import English

parser = English()
def tokenize(text):
    lda_tokens = []
    tokens = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.like_url:
            continue
        elif token.orth_.startswith('@'):
            continue
        elif not token.orth_.isalpha():
            continue
        else:
            lda_tokens.append(token.lower_)
    return lda_tokens

In [12]:
import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet as wn
def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma
    
from nltk.stem.wordnet import WordNetLemmatizer
def get_lemma2(word):
    return WordNetLemmatizer().lemmatize(word)

[nltk_data] Downloading package wordnet to /Users/wky/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [40]:
# nltk.download('stopwords')
en_stop = set(nltk.corpus.stopwords.words('english'))
en_stop.update([
    'family', 
    'domestic',
    'violence',
    'one', 
    'amp',
    'get',
    'women',
    'woman'
])

In [41]:
def prepare_text_for_lda(text):
    tokens = tokenize(text)
#     tokens = [token for token in tokens if len(token) > 4]
    tokens = [token for token in tokens if token not in en_stop]
    tokens = [get_lemma(token) for token in tokens]
    return tokens

In [42]:
text_data = []
for record in records:
    text = record['doc']['text']
    tokens = prepare_text_for_lda(text)
    text_data.append(tokens)

In [43]:
from gensim import corpora
dictionary = corpora.Dictionary(text_data)
corpus = [dictionary.doc2bow(text) for text in text_data]

import pickle
pickle.dump(corpus, open('output/corpus.pkl', 'wb'))
dictionary.save('output/dictionary.gensim')

In [44]:
import gensim

NUM_TOPICS = 5
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
ldamodel.save('output/model5.gensim')
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

(0, '0.023*"rosie" + 0.022*"batty" + 0.012*"forum" + 0.012*"year"')
(1, '0.014*"election" + 0.014*"problem" + 0.014*"rosie" + 0.014*"batty"')
(2, '0.023*"see" + 0.016*"phil" + 0.016*"terror" + 0.009*"child"')
(3, '0.018*"new" + 0.010*"say" + 0.010*"need" + 0.010*"reason"')
(4, '0.016*"fund" + 0.016*"batty" + 0.016*"wrong" + 0.011*"kill"')


In [46]:
dictionary = gensim.corpora.Dictionary.load('output/dictionary.gensim')
corpus = pickle.load(open('output/corpus.pkl', 'rb'))
lda = gensim.models.ldamodel.LdaModel.load('output/model5.gensim')

import pyLDAvis.gensim
lda_display = pyLDAvis.gensim.prepare(lda, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display)

/anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))
